# Generador de Videos

Se importan todas las librerías
* cv2 para manipular imágenes y generar los frames del video
* numpy para manejar arreglos y normalizar la energía del audio
* gTTS para convertir texto a voz
* subprocess para ejecutar comandos de ffmpeg
* wave y audioop para analizar la energía del audio
* imageio_ffmpeg para obtener la ruta del ejecutable ffmpeg
* re para limpiar y procesar los textos de los libretos.

In [1]:
# Librerías
import cv2
import numpy as np
from gtts import gTTS
import subprocess
import os
import wave
import audioop
import imageio_ffmpeg
import re

Se definen las carpetas donde se encuentran los libretos y las imágenes que se usarán para animar la boca, así como la carpeta de salida donde se guardarán los videos finales. También se crea automáticamente la carpeta de salida si no existe, se define el rango de años a procesar, y se obtiene la ruta del ejecutable de ffmpeg para poder usarlo en la generación de los archivos de video y audio.

In [3]:
# In carpetas
libretos_folder = "libretos"
imagenes_folder = "imagenes"
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

años = list(range(1985, 2025))  # 1985 a 2024

ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()

Se definen dos funciones principales
* limpiar_texto: normaliza el texto de los libretos eliminando comillas, puntos suspensivos y espacios extras para que la conversión a voz sea más limpia
* generar_video crea un video por año combinando la narración generada a partir del texto con las imágenes que simulan la boca abierta y cerrada. La función analiza la energía del audio para decidir qué imagen mostrar en cada frame, genera un video temporal, lo sincroniza con el audio y finalmente guarda el resultado en la carpeta de salida. Al final, se limpia todo archivo temporal y se repite el proceso para cada año en el rango definido.

In [5]:
# Funciones de generación de video
def limpiar_texto(texto):
    texto = texto.replace('"', '').replace("'", "")       
    texto = texto.replace("…", "")                        
    texto = re.sub(r'\s+', ' ', texto)                
    return texto.strip()

def generar_video(año):
    txt_path = os.path.join(libretos_folder, f"{año}.txt")
    abierta_path = os.path.join(imagenes_folder, f"{año}_abierta.png")
    cerrada_path = os.path.join(imagenes_folder, f"{año}_cerrada.png")
    output_path = os.path.join(output_folder, f"{año}.mp4")
    
    if not os.path.exists(txt_path):
        print(f"❌ Libreto del año {año} no encontrado.")
        return
    with open(txt_path, "r", encoding="utf-8") as f:
        texto = limpiar_texto(f.read())
    
    voz_mp3 = f"temp_voz_{año}.mp3"
    voz_wav = f"temp_voz_{año}.wav"
    gTTS(texto, lang="es").save(voz_mp3)
    subprocess.run([ffmpeg_path, "-y", "-i", voz_mp3, voz_wav, "-loglevel", "quiet"])
    
    with wave.open(voz_wav, 'rb') as w:
        frames = w.readframes(-1)
        rate = w.getframerate()
        segment_size = rate // 10
        energy = [audioop.rms(frames[i:i+segment_size], 2) for i in range(0, len(frames), segment_size)]
    energy = np.interp(energy, (min(energy), max(energy)), (0, 1))
    
    img_open = cv2.imread(abierta_path)
    img_closed = cv2.imread(cerrada_path)
    if img_open is None or img_closed is None:
        print(f"❌ Imágenes del año {año} no encontradas.")
        return
    alto, ancho, _ = img_open.shape
    
    fps = 10
    num_frames = len(energy)
    video_temp = f"temp_video_{año}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_temp, fourcc, fps, (ancho, alto))
    
    for i in range(num_frames):
        idx = int((i / num_frames) * len(energy))
        frame = img_open if energy[idx] > 0.3 else img_closed
        out.write(frame)
    out.release()
    
    subprocess.run([
        ffmpeg_path, "-y",
        "-i", video_temp,
        "-i", voz_mp3,
        "-c:v", "copy",
        "-c:a", "aac",
        "-shortest",
        output_path
    ])
    
    os.remove(voz_mp3)
    os.remove(voz_wav)
    os.remove(video_temp)
    
    print(f"✅ Video del año {año} generado: {output_path}")

for año in años:
    generar_video(año)

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/Volumes/tempdisk/sw --extra-cflags=-fno-stack-check --arch=arm64 --cc=/usr/bin/clang --enable-gpl --enable-libvmaf --enable-libopenjpeg --enable-libopus --enable-libmp3lame --enable-libx264 --enable-libx265 --enable-libvpx --enable-libwebp --enable-libass --enable-libfreetype --enable-fontconfig --enable-libtheora --enable-libvorbis --enable-libsnappy --enable-libaom --enable-libvidstab --enable-libzimg --enable-libsvtav1 --enable-libharfbuzz --enable-libkvazaar --pkg-config-flags=--static --enable-ffplay --enable-postproc --enable-neon --enable-runtime-cpudetect --disable-indev=qtkit --disable-indev=x11grab_xcb
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.100 / 61. 19.100
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100 / 10.  4.100
  libswscale      8. 

✅ Video del año 1985 generado: output/1985.mp4


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/Volumes/tempdisk/sw --extra-cflags=-fno-stack-check --arch=arm64 --cc=/usr/bin/clang --enable-gpl --enable-libvmaf --enable-libopenjpeg --enable-libopus --enable-libmp3lame --enable-libx264 --enable-libx265 --enable-libvpx --enable-libwebp --enable-libass --enable-libfreetype --enable-fontconfig --enable-libtheora --enable-libvorbis --enable-libsnappy --enable-libaom --enable-libvidstab --enable-libzimg --enable-libsvtav1 --enable-libharfbuzz --enable-libkvazaar --pkg-config-flags=--static --enable-ffplay --enable-postproc --enable-neon --enable-runtime-cpudetect --disable-indev=qtkit --disable-indev=x11grab_xcb
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.100 / 61. 19.100
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100 / 10.  4.100
  libswscale      8. 

✅ Video del año 1986 generado: output/1986.mp4


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/Volumes/tempdisk/sw --extra-cflags=-fno-stack-check --arch=arm64 --cc=/usr/bin/clang --enable-gpl --enable-libvmaf --enable-libopenjpeg --enable-libopus --enable-libmp3lame --enable-libx264 --enable-libx265 --enable-libvpx --enable-libwebp --enable-libass --enable-libfreetype --enable-fontconfig --enable-libtheora --enable-libvorbis --enable-libsnappy --enable-libaom --enable-libvidstab --enable-libzimg --enable-libsvtav1 --enable-libharfbuzz --enable-libkvazaar --pkg-config-flags=--static --enable-ffplay --enable-postproc --enable-neon --enable-runtime-cpudetect --disable-indev=qtkit --disable-indev=x11grab_xcb
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.100 / 61. 19.100
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100 / 10.  4.100
  libswscale      8. 

...

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/Volumes/tempdisk/sw --extra-cflags=-fno-stack-check --arch=arm64 --cc=/usr/bin/clang --enable-gpl --enable-libvmaf --enable-libopenjpeg --enable-libopus --enable-libmp3lame --enable-libx264 --enable-libx265 --enable-libvpx --enable-libwebp --enable-libass --enable-libfreetype --enable-fontconfig --enable-libtheora --enable-libvorbis --enable-libsnappy --enable-libaom --enable-libvidstab --enable-libzimg --enable-libsvtav1 --enable-libharfbuzz --enable-libkvazaar --pkg-config-flags=--static --enable-ffplay --enable-postproc --enable-neon --enable-runtime-cpudetect --disable-indev=qtkit --disable-indev=x11grab_xcb
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.100 / 61. 19.100
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100 / 10.  4.100
  libswscale      8. 

✅ Video del año 2022 generado: output/2022.mp4


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/Volumes/tempdisk/sw --extra-cflags=-fno-stack-check --arch=arm64 --cc=/usr/bin/clang --enable-gpl --enable-libvmaf --enable-libopenjpeg --enable-libopus --enable-libmp3lame --enable-libx264 --enable-libx265 --enable-libvpx --enable-libwebp --enable-libass --enable-libfreetype --enable-fontconfig --enable-libtheora --enable-libvorbis --enable-libsnappy --enable-libaom --enable-libvidstab --enable-libzimg --enable-libsvtav1 --enable-libharfbuzz --enable-libkvazaar --pkg-config-flags=--static --enable-ffplay --enable-postproc --enable-neon --enable-runtime-cpudetect --disable-indev=qtkit --disable-indev=x11grab_xcb
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.100 / 61. 19.100
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100 / 10.  4.100
  libswscale      8. 

✅ Video del año 2023 generado: output/2023.mp4
✅ Video del año 2024 generado: output/2024.mp4


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/Volumes/tempdisk/sw --extra-cflags=-fno-stack-check --arch=arm64 --cc=/usr/bin/clang --enable-gpl --enable-libvmaf --enable-libopenjpeg --enable-libopus --enable-libmp3lame --enable-libx264 --enable-libx265 --enable-libvpx --enable-libwebp --enable-libass --enable-libfreetype --enable-fontconfig --enable-libtheora --enable-libvorbis --enable-libsnappy --enable-libaom --enable-libvidstab --enable-libzimg --enable-libsvtav1 --enable-libharfbuzz --enable-libkvazaar --pkg-config-flags=--static --enable-ffplay --enable-postproc --enable-neon --enable-runtime-cpudetect --disable-indev=qtkit --disable-indev=x11grab_xcb
  libavutil      59. 39.100 / 59. 39.100
  libavcodec     61. 19.100 / 61. 19.100
  libavformat    61.  7.100 / 61.  7.100
  libavdevice    61.  3.100 / 61.  3.100
  libavfilter    10.  4.100 / 10.  4.100
  libswscale      8. 